# 📘 **Text Representation | NLP Lecture 4**

### Topics: Bag of Words | TF-IDF | N-grams (Uni-grams, Bi-grams)


## I. **Introduction to Text Representation**

**Goal:** Convert text data into numerical form (a process also known as *Text Vectorisation* or *Feature Extraction from Text*) so that ML algorithms can process it.

### 🔹 Importance

* **Feature Quality:** The effectiveness of ML models heavily depends on feature quality — *“Garbage in, garbage out.”*
* **NLP Pipeline:** Follows data acquisition and pre-processing; crucial for ML-based NLP.
* **Objective:** Numerical representation should capture the *semantic meaning* of text.

### 🔹 Challenges

* Text → Numbers is hard (unlike image or audio data).
* Requires intelligent mapping of linguistic meaning to mathematical form.

### 🔹 Techniques Covered

1. One-Hot Encoding
2. Bag of Words (BoW)
3. N-grams (Uni, Bi, Tri-grams)
4. TF–IDF (Term Frequency–Inverse Document Frequency)
5. Custom Features
   *(Future: Word Embeddings – Word2Vec, GloVe, etc.)*

---

## II. **Key Terminology**

| Term                     | Description                                      |
| ------------------------ | ------------------------------------------------ |
| **Corpus (C)**           | All words from all documents (including repeats) |
| **Vocabulary (V)**       | Set of *unique* words from the corpus            |
| **Document (D)**         | A single text unit (sentence, paragraph, etc.)   |
| **Word (W) or Term (T)** | A single token in a document                     |

---

#### **1. One-Hot Encoding (1950s–1960s, very early NLP)**

* **Idea:** Represent each word as a binary vector — all 0s except for a single 1 in the position corresponding to that word in the vocabulary.
* **Example:**
  Vocabulary: ["cat", "dog", "fish"]
  “dog” → [0, **1**, 0]
* **Limitation:**

  * Ignores context and meaning
  * High dimensional and sparse
- ⚠️ Drawbacks

🔹 Disadvantages

- ❌ Sparsity: Huge, mostly-zero matrices.
- ❌ Non-fixed size: Different document lengths = different input sizes.
- ❌ OOV(Out-Of-Vocabulary) problem: New words can’t be represented.
- ❌ No semantics: “walk” and “run” are equally distant from “bottle”.


In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

corpus = ["dog barks", "cat meows", "dog runs"]
words = list(set(" ".join(corpus).split()))  # unique vocab
print("Vocabulary:", words)

# Manual one-hot encoding
import numpy as np
V = len(words)
encoding = {word: np.eye(V)[i] for i, word in enumerate(words)}
encoding


---

#### **2. Bag of Words (BoW) (1980s–1990s)**

* **Idea:** Represent a document by word **counts** (or frequencies) — order of words is ignored.

- Vocabulary from corpus → Each document = vector of word frequencies.
- Ignores order → treats text as a bag of words.
- Used widely for text classification.

* **Example:**
  “Dog bites man” and “Man bites dog” → same vector (same counts).
* **Limitation:**

#### 🔹 Advantages

- ✅ Fixed-size vector for any document
- ✅ Tolerates unseen words (ignored at inference)/OOV words ignored gracefully

#### 🔹 Disadvantages

- ❌ Sparse vectors/Large vocabulary → sparse vectors
- ❌ Loses word order/Ignores word order/context (syntax)
- ❌ Fails with negation — “good” vs. “not good” appear similar


* 🔹 Scikit-learn:
- CountVectorizer()

* 🔹 Key Parameters
  - binary=True: presence (1/0) instead of count.
  - max_features: limit vocabulary to top-N frequent words.
  - stop_words: remove common stopwords.


In [2]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
corpus = [
    "The cat sat on the mat",
    "The dog sat on the log",
    "The cat chased the mouse"
]

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

print("Vocabulary:", vectorizer.get_feature_names_out())
print(pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out()))

Vocabulary: ['cat' 'chased' 'dog' 'log' 'mat' 'mouse' 'on' 'sat' 'the']
   cat  chased  dog  log  mat  mouse  on  sat  the
0    1       0    0    0    1      0   1    1    2
1    0       0    1    1    0      0   1    1    2
2    1       1    0    0    0      1   0    0    2


In [3]:
# ⚙️ Key Parameters
vectorizer1 = CountVectorizer(
    binary=True,             # presence/absence instead of counts
    max_features=10,         # limit vocabulary size
    stop_words='english',    # remove stopwords
)

In [4]:
vectorizer1.fit_transform(corpus).toarray()

array([[1, 0, 0, 0, 1, 0, 1],
       [0, 0, 1, 1, 0, 0, 1],
       [1, 1, 0, 0, 0, 1, 0]])

#### **3. N-grams (Unigram, Bigram, Trigram) (1990s–2000s)**

* **Idea:** Capture **local word order** by looking at sequences of *n* words.
## * 📘 Concept

- Extends BoW by including sequences of N words.
- Captures local word order and context.

* **Examples:**

  * Unigrams: “dog”, “bites”, “man”
  * Bigrams: “dog bites”, “bites man”
  * Trigrams: “dog bites man”
* **Benefit:** Adds some context awareness.
- Captures context and short phrases

- Helps handle negations and idioms

* **Limitation:** Still sparse, grows combinatorially with *n*.
  - Vocabulary size grows fast (computational cost)
  - Still sparse and OOV issues remain

In [ ]:
# Bi-gram example
bi_vectorizer = CountVectorizer(ngram_range=(2,2))
# (2, 2): Setting the range from 2 (minimum n-value) to 2 (maximum n-value)
# so (2,2 means the vectorizer will only consider sequences of exactly two words, known as bigrams.
X_bi = bi_vectorizer.fit_transform(corpus)
corpus = [
    "The cat sat on the mat",
    "The dog sat on the log",
    "The cat chased the mouse"
]
print("Bi-gram Vocabulary:", bi_vectorizer.get_feature_names_out())
print(pd.DataFrame(X_bi.toarray(), columns=bi_vectorizer.get_feature_names_out()))
print("==============================")
print(len(bi_vectorizer.get_feature_names_out()))

Bi-gram Vocabulary: ['cat' 'cat chased' 'cat chased the' 'cat sat' 'cat sat on' 'chased'
 'chased the' 'chased the mouse' 'dog' 'dog sat' 'dog sat on' 'log' 'mat'
 'mouse' 'on' 'on the' 'on the log' 'on the mat' 'sat' 'sat on'
 'sat on the' 'the' 'the cat' 'the cat chased' 'the cat sat' 'the dog'
 'the dog sat' 'the log' 'the mat' 'the mouse']
   cat  cat chased  cat chased the  cat sat  cat sat on  chased  chased the  \
0    1           0               0        1           1       0           0   
1    0           0               0        0           0       0           0   
2    1           1               1        0           0       1           1   

   chased the mouse  dog  dog sat  ...  sat on the  the  the cat  \
0                 0    0        0  ...           1    2        1   
1                 0    1        1  ...           1    2        0   
2                 1    0        0  ...           0    2        1   

   the cat chased  the cat sat  the dog  the dog sat  the log  t

In [ ]:
# Uni + Bi + Tri-grams
combo_vectorizer = CountVectorizer(ngram_range=(1,3))
X_combo = combo_vectorizer.fit_transform(corpus)
print("Combined Vocabulary Size:", len(combo_vectorizer.get_feature_names_out()))

print(X_combo)

---

#### **4. TF–IDF (Term Frequency–Inverse Document Frequency) (1990s–2000s)**

* **Idea:** Weigh words by importance — frequent in a document but rare across the corpus.
- Assigns importance weights to words instead of raw counts.
* **Formula:**
  TF-IDF(T,D)=TF(T,D)×IDF(T)

- TF= Word occurrences in doc/Total words in doc'

- Inverse Document Frequency (IDF):
- IDF=log(N/n_T)

Weight = TF × IDF
* **Benefit:** Reduces impact of common words like “the”, “and”.
* **Limitation:** Still based on counts, no semantic understanding.

* ✅ Advantages

Reduces weight of common words
Useful for information retrieval (e.g., search engines)

* ❌ Disadvantages

- Sparse matrix
- OOV issue
- Still no deep semantic relation captured

---

![](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*XeYB3-C2cD3KIi975ql4_g.png)

![](https://miro.medium.com/v2/resize:fit:720/format:webp/1*9tecGbVZZ8wxniSPoiXcYA.png)

https://medium.com/@abhishekjainindore24/tf-idf-in-nlp-term-frequency-inverse-document-frequency-e05b65932f1d

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
    "The cat sat on the mat",
    "The dog sat on the log",
    "The cat chased the mouse"
]

tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(corpus)

print("Vocabulary:", tfidf.get_feature_names_out())
print(pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names_out()))


Vocabulary: ['cat' 'chased' 'dog' 'log' 'mat' 'mouse' 'on' 'sat' 'the']
        cat    chased       dog       log       mat     mouse        on  \
0  0.374207  0.000000  0.000000  0.000000  0.492038  0.000000  0.374207   
1  0.000000  0.000000  0.468699  0.468699  0.000000  0.000000  0.356457   
2  0.381519  0.501651  0.000000  0.000000  0.000000  0.501651  0.000000   

        sat       the  
0  0.374207  0.581211  
1  0.356457  0.553642  
2  0.000000  0.592567  


#### **5. Word Embeddings (Word2Vec, GloVe, FastText, etc.) (2013 onward)**

* **Idea:** Learn **dense, low-dimensional vectors** where similar words are close in vector space.
* **Example:**
  Vector(“king”) – Vector(“man”) + Vector(“woman”) ≈ Vector(“queen”)
* **Benefit:** Captures **semantic meaning** and **relationships** between words.
* **Limitation:** Fixed for each word — context-independent.

---

https://www.cs.cmu.edu/~dst/WordEmbeddingDemo/tutorial.html

# 🧠 **1. Word2Vec**

### 📘 **Definition**

**Word2Vec** (by Google, 2013) is a **neural embedding model** that learns to represent words as dense vectors.
It uses two main architectures:

* **CBOW (Continuous Bag of Words):** Predicts a word based on its context.
* **Skip-Gram:** Predicts context words given a target word.

These embeddings capture **semantic and syntactic relationships** between words — e.g.,
`vector("king") - vector("man") + vector("woman") ≈ vector("queen")`.

---

### ✅ **Advantages**

* Captures both **semantic** and **syntactic** relationships.
* Trains efficiently on large datasets.
* Performs well in many NLP downstream tasks.

### ❌ **Disadvantages**

* Doesn’t handle **out-of-vocabulary (OOV)** words.
* Ignores **subword (morphological)** information.
* Embeddings depend heavily on training data quality.


In [3]:
### 💻 **Code Example**
!pip install gensim

  Using cached wrapt-2.0.1-cp310-cp310-win_amd64.whl.metadata (9.2 kB)
   ---------------------------------------- 0.0/24.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.4 MB ? eta -:--:--
   - -------------------------------------- 1.0/24.4 MB 12.7 MB/s eta 0:00:02
   - -------------------------------------- 1.0/24.4 MB 12.7 MB/s eta 0:00:02
   ------ --------------------------------- 3.7/24.4 MB 6.4 MB/s eta 0:00:04
   ------------ --------------------------- 7.6/24.4 MB 9.6 MB/s eta 0:00:02
   --------------- ------------------------ 9.4/24.4 MB 10.9 MB/s eta 0:00:02
   --------------- ------------------------ 9.4/24.4 MB 10.9 MB/s eta 0:00:02
   --------------- ------------------------ 9.4/24.4 MB 10.9 MB/s eta 0:00:02
   ---------------- ----------------------- 10.2/24.4 MB 6.0 MB/s eta 0:00:03
   ----------------------- ---------------- 14.4/24.4 MB 7.4 MB/s eta 0:00:02
   -------------------------------- ------- 19.9/24.4 MB 9.3 MB/s eta 0:00:01
   --------

In [1]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

# Sample corpus
sentences = [
    "I love deep learning and natural language processing",
    "Word embeddings are useful for NLP tasks",
    "Word2Vec is a great model for learning word vectors"
]
# Preprocessing
tokenized_sentences = [simple_preprocess(sentence) for sentence in sentences]

tokenized_sentences

ModuleNotFoundError: No module named 'gensim'

In [2]:
# Train Word2Vec model
model_w2v = Word2Vec(sentences=tokenized_sentences, vector_size=100, window=5, min_count=1, sg=1)
 # vector_size=100: The size of the dense vector (embedding) for each word, determining its features.
# window=5: The maximum distance between the target word and context words (5 words to the left and 5 to the right).
# min_count=1: Ignores all words with a total frequency lower than 1 (i.e., includes every word that appears at least once).
# sg=1: Specifies the Skip-Gram training algorithm, which predicts context words from a target word (better for infrequent words). (Use sg=0 for CBOW - Continuous Bag of Words).

# Check vector and similar words
print(model_w2v.wv["learning"])
print(model_w2v.wv.most_similar("learning"))

NameError: name 'Word2Vec' is not defined

# 📗 **2. GloVe (Global Vectors for Word Representation)**

### 📘 **Definition**

**GloVe** (by Stanford, 2014) learns word embeddings by analyzing **global word co-occurrence statistics** across the entire corpus.
It focuses on how often words appear together — building a **co-occurrence matrix**, then factorizing it to learn embeddings.

---

### ✅ **Advantages**

* Captures **global context** better than Word2Vec (which is local).
* Produces **consistent** embeddings using statistical information.
* Pretrained models available (trained on huge corpora like Wikipedia).

### ❌ **Disadvantages**

* Doesn’t handle **OOV** words.
* Cannot learn new embeddings once pretrained.
* Needs large memory for co-occurrence matrix.

---

### 💻 **Code Example**

```python
# !pip install torchtext torch

from torchtext.vocab import GloVe
import torch

# Load pretrained GloVe embeddings (50 dimensions)
glove = GloVe(name="6B", dim=50)

# Get vector for a word
word_vec = glove["computer"]
print(word_vec)

# Compute similarity
sim = torch.cosine_similarity(glove["king"].unsqueeze(0), glove["queen"].unsqueeze(0))
print(f"Similarity(king, queen): {sim.item():.4f}")
```

---

# 📘 **3. FastText**

### 📘 **Definition**

**FastText** (by Facebook, 2016) extends Word2Vec by representing each word as a **bag of character n-grams**.
This allows it to understand **morphology** and generate embeddings for unseen words.

Example:
`"playing"` → `["pla", "lay", "ayi", "yin", "ing"]`

---

### ✅ **Advantages**

* Handles **out-of-vocabulary** (OOV) words.
* Captures **subword information** (prefixes, suffixes).
* Works well for **morphologically rich languages** (like German or Turkish).

### ❌ **Disadvantages**

* Slightly **slower** to train than Word2Vec.
* Requires more memory.
* Subword info might not always improve performance (for small datasets).

---

### 💻 **Code Example**

```python
from gensim.models import FastText
from gensim.utils import simple_preprocess

# Corpus
sentences = [
    "I love machine learning and data science",
    "FastText creates embeddings using subwords",
    "It can handle unseen words like learnings"
]

tokenized_sentences = [simple_preprocess(s) for s in sentences]

# Train FastText model
model_ft = FastText(sentences=tokenized_sentences, vector_size=100, window=5, min_count=1, workers=4)

# Vector for a known word
print(model_ft.wv["learning"])

# Vector for an unseen (OOV) word
print(model_ft.wv["learnings"])  # Works!
```






---

# 📊 **🔍 Summary Table**

| Model        | Year | Creator  | Key Idea                    | Handles OOV | Pros                              | Cons                       |
| ------------ | ---- | -------- | --------------------------- | ----------- | --------------------------------- | -------------------------- |
| **Word2Vec** | 2013 | Google   | Predictive (CBOW/Skip-Gram) | ❌           | Semantic relationships, efficient | No OOV, ignores morphology |
| **GloVe**    | 2014 | Stanford | Global co-occurrence matrix | ❌           | Global context, pretrained models | Static, no OOV             |
| **FastText** | 2016 | Facebook | Subword (character n-grams) | ✅           | Handles OOV, morphological info   | Slower, more memory        |

---

Would you like me to add a **visualization (PCA or t-SNE)** to compare how similar words cluster across these models?


In [ ]:
# https://projector.tensorflow.org/



#### **6. Contextual Word Embeddings (BERT, GPT, etc.) (2018 onward)**

* **Idea:** Represent words **in context**, so “bank” in “river bank” ≠ “bank” in “money bank”.
* **Examples:** ELMo (2018), BERT (2018), GPT series (2018+)
* **Benefit:** State-of-the-art performance across NLP tasks.
* **Limitation:** Computationally expensive.


# 🧠 **Contextual Word Embeddings | NLP Lecture 5**

---

## I. **Introduction**

Traditional techniques like **One-Hot**, **BoW**, **N-grams**, and **TF–IDF** treat each word as **independent** and **context-free**.
They fail to capture:

* **Meaning differences** based on context (e.g., *bank* = river bank vs. money bank)
* **Semantic similarity** between related words (*good*, *great*, *excellent*)

➡️ **Contextual Word Embeddings** solve these issues using **deep learning**.

---

## II. **From Static to Contextual Embeddings**

### 1. **Static Embeddings (Word2Vec, GloVe, FastText)**

* Each word has **one fixed vector**, no matter where it appears.
* “bank” → same vector in both “river bank” and “bank loan”.
* Captures general *semantic similarity*, but **no context**.

### 2. **Contextual Embeddings (ELMo, BERT, GPT, etc.)**

* Word meaning changes **depending on its sentence context**.
* “bank” in “river bank” and “bank loan” → **different embeddings**.
* Generated dynamically by **transformer-based** neural networks.

---

## III. **How Contextual Embeddings Work**

### 🔹 **Architecture**

Most modern embeddings are based on the **Transformer architecture**, introduced in *“Attention is All You Need” (Vaswani et al., 2017)*.

Key idea:
👉 Use **Self-Attention** to learn relationships between all words in a sentence.

### 🔹 **Self-Attention**

* Every word looks at all other words to understand its context.
* Example:

  * Sentence: *“The bank of the river was flooded.”*
  * The model looks at “river” → understands that *bank* refers to a geographical feature.

---

## IV. **Popular Contextual Embedding Models**

| Model               | Type                  | Key Idea                              | Notes                        |
| ------------------- | --------------------- | ------------------------------------- | ---------------------------- |
| **ELMo (2018)**     | BiLSTM                | Contextual from both directions       | First major contextual model |
| **BERT (2019)**     | Transformer           | Bidirectional context using attention | Widely used in NLP tasks     |
| **GPT (2018–2024)** | Transformer (decoder) | Predicts next word (left-to-right)    | Used for generation tasks    |
| **RoBERTa**         | BERT variant          | More training, better performance     |                              |
| **DistilBERT**      | Lightweight BERT      | 40% smaller, similar accuracy         |                              |

---

## V. **Example: Using BERT Embeddings**

### 💻 **Code Example (Using `transformers` library)**


![](https://miro.medium.com/v2/resize:fit:1200/1*4bg3WZRcZMYFfler0StxXQ.png)

In [ ]:
# https://medium.com/@lmpo/a-brief-history-of-lmms-from-transformers-2017-to-deepseek-r1-2025-dae75dd3f59a

🔹 Example 1 — BERT (Bidirectional Encoder Representations from Transformers)


In [ ]:
# !pip install transformers torch

from transformers import BertTokenizer, BertModel
import torch

# Load pretrained model and tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

# Example sentences
sentence = "I went to the bank to withdraw money."
inputs = tokenizer(sentence, return_tensors="pt")

# Get embeddings
with torch.no_grad():
    outputs = model(**inputs)

# Extract last hidden states (contextual embeddings)
last_hidden_states = outputs.last_hidden_state
print(last_hidden_states.shape)  # [batch_size, sequence_length, hidden_size]

# Example: get embedding for "bank"
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
print(tokens)
bank_index = tokens.index("bank")
print(f"Embedding for 'bank':\n", last_hidden_states[0, bank_index, :5])  # first 5 dims


🔹 Example 2 — GPT (Generative Pre-trained Transformer)

In [ ]:
from transformers import GPT2Tokenizer, GPT2Model

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2Model.from_pretrained("gpt2")

sentence = "The river bank was full of trees."
inputs = tokenizer(sentence, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)

# Get last hidden states
embeddings = outputs.last_hidden_state
print(embeddings.shape)

# Inspect token-level embeddings
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
print(tokens)

🔹 Example 3 — Compare Contexts for “bank”

In [ ]:
sentences = [
    "I deposited money in the bank.",
    "We sat by the river bank and watched the water."
]

for sentence in sentences:
    inputs = tokenizer(sentence, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    bank_index = tokens.index("bank")
    embedding = outputs.last_hidden_state[0, bank_index, :]
    print(f"Sentence: {sentence}")
    print(f"Bank embedding (first 5 dims): {embedding[:5]}")
    print("-" * 50)


| Model    | Year       | Architecture        | Context Type  | Pros                         | Cons                       |
| -------- | ---------- | ------------------- | ------------- | ---------------------------- | -------------------------- |
| **ELMo** | 2018       | Bi-LSTM             | Bidirectional | Context-aware, simple        | Slower, older architecture |
| **BERT** | 2018       | Transformer Encoder | Bidirectional | Best for understanding tasks | Heavy, non-generative      |
| **GPT**  | 2018–2023+ | Transformer Decoder | Left-to-right | Generative power, flexible   | Unidirectional context     |




```python
!pip install transformers torch --quiet

from transformers import BertTokenizer, BertModel
import torch

# Load pretrained BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Example sentence
sentence = "The bank of the river was flooded."

# Tokenize input
inputs = tokenizer(sentence, return_tensors='pt')

# Get embeddings
with torch.no_grad():
    outputs = model(**inputs)

# outputs contains:
# - last_hidden_state: embeddings for each token
# - pooler_output: embedding for entire sentence (CLS token)
word_embeddings = outputs.last_hidden_state
sentence_embedding = outputs.pooler_output

print("Word Embeddings Shape:", word_embeddings.shape)
print("Sentence Embedding Shape:", sentence_embedding.shape)
```

### 🧩 **Output Explanation**

* `word_embeddings`: Tensor of shape `[1, num_tokens, 768]` → one 768-dim vector per token.
* `sentence_embedding`: `[1, 768]` → condensed representation of entire sentence.

---

## VI. **Understanding the Embedding Dimensions**

| Model           | Vector Size | Layers | Context Type         |
| --------------- | ----------- | ------ | -------------------- |
| **BERT-base**   | 768         | 12     | Bidirectional        |
| **BERT-large**  | 1024        | 24     | Bidirectional        |
| **GPT-2 small** | 768         | 12     | Unidirectional       |
| **ELMo**        | 1024        | 2      | Bidirectional (LSTM) |

Each embedding dimension encodes complex linguistic patterns such as syntax, semantics, and relationships.

---

## VII. **Sentence-Level Embeddings**

Instead of token-level vectors, sometimes we need **whole-sentence embeddings** (for tasks like similarity or classification).

### 🔹 Options:

1. **[CLS] Token (BERT):**
   Use the first token embedding (sentence representation).
2. **Mean Pooling:**
   Average over all token embeddings.
3. **Sentence-BERT (SBERT):**
   Fine-tuned version of BERT for **semantic similarity**.

### 💻 Example (SBERT)

```python
!pip install sentence-transformers --quiet
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
sentences = ["The cat sat on the mat.", "A dog lay on the rug."]

embeddings = model.encode(sentences)

print("Embedding Shape:", embeddings.shape)
```

➡️ Output: `(2, 384)` — Each sentence = 384-dimensional dense vector.
You can then compute **cosine similarity** to measure semantic closeness.

---

## VIII. **Applications of Contextual Embeddings**

| Task                               | Use                                         |
| ---------------------------------- | ------------------------------------------- |
| **Text Classification**            | Sentiment, topic, emotion detection         |
| **Named Entity Recognition (NER)** | Extracting entities like names, dates, etc. |
| **Question Answering**             | Understanding context in questions          |
| **Semantic Search**                | Finding meaning-based matches               |
| **Machine Translation**            | Context-aware language translation          |
| **Chatbots & Summarization**       | Contextual understanding of input text      |

---

## IX. **Advantages vs. Traditional Methods**

| Feature                | Traditional (TF-IDF / BoW) | Contextual (BERT / GPT)                 |
| ---------------------- | -------------------------- | --------------------------------------- |
| Word meaning           | Fixed                      | Context-aware                           |
| Vocabulary handling    | OOV issue                  | Handles unseen words via subword tokens |
| Sparsity               | High                       | Dense                                   |
| Training need          | None (handcrafted)         | Pretrained deep models                  |
| Computation            | Lightweight                | Heavy but powerful                      |
| Semantic understanding | Weak                       | Strong                                  |

---

## X. **Key Takeaways**

* **Contextual embeddings** revolutionized NLP by enabling models to *understand meaning in context*.
* They are **dense, continuous, and dynamic** representations.
* Models like **BERT, GPT, RoBERTa, and SBERT** form the backbone of modern NLP systems.
* These embeddings power applications like ChatGPT, semantic search, and summarization.

---

## XI. **Practice Exercise**

1. Load a small text dataset (e.g., IMDb reviews).
2. Generate:

   * TF-IDF vectors
   * BERT sentence embeddings
3. Compare similarity between sentences using cosine similarity.
4. Visualize embeddings with **PCA or t-SNE**.

---



---

### 🧭 **Summary Timeline**

| Era         | Technique                         | Type            | Key Idea                   |
| ----------- | --------------------------------- | --------------- | -------------------------- |
| 1950s–1960s | One-Hot Encoding                  | Sparse          | Binary identity vectors    |
| 1980s–1990s | Bag of Words (BoW)                | Sparse          | Count-based, ignores order |
| 1990s–2000s | N-grams                           | Sparse          | Adds local context         |
| 1990s–2000s | TF–IDF                            | Weighted Sparse | Importance weighting       |
| 2013+       | Word Embeddings (Word2Vec, GloVe) | Dense           | Semantic similarity        |
| 2018+       | Contextual Embeddings (BERT, GPT) | Dense           | Meaning depends on context |

---
